# TP1

In [16]:
import pandas as pd

def loadData(file):
    # Charger les données CSV en spécifiant les noms de colonnes
    data = pd.read_csv(file, index_col=3,  
                       names=['gender', 'genderLooking', 'age', 'name', 
                              'counts_details', 'counts_pictures', 'counts_profileVisits', 'counts_kisses', 
                              'counts_fans', 'counts_g', 
                              'flirtInterests_chat', 'flirtInterests_friends', 'flirtInterests_date', 
                              'country', 'city', 'location', 'distance', 'isFlirtstar', 'isHighlighted', 
                              'isInfluencer', 'isMobile', 'isNew', 'isOnline', 'isVip', 'lang_count', 
                              'lang_fr', 'lang_en', 'lang_de', 'lang_it', 'lang_es', 'lang_pt', 'verified', 
                              'shareProfileEnabled', 'lastOnlineDate', 'lastOnlineTime', 'birthd', 'crypt', 
                              'freetext', 'whazzup', 'userId', 'pictureId', 'isSystemProfile'])
    
    # Convertir les colonnes pertinentes en numériques, en traitant les erreurs avec NaN si nécessaire
    data['counts_profileVisits'] = pd.to_numeric(data['counts_profileVisits'], errors='coerce')
    data['counts_kisses'] = pd.to_numeric(data['counts_kisses'], errors='coerce')
                
    return data


# Charger les données depuis le fichier CSV
dataset = loadData("lovoo_v3_users_api-results.csv")

# Colonnes à supprimer du DataFrame
colonnes_a_supprimer = ['age', 'gender', 'genderLooking', 'counts_details', 'counts_pictures', 'counts_fans', 'counts_g',
                        'flirtInterests_chat', 'flirtInterests_friends', 'flirtInterests_date', 
                        'country', 'city', 'location', 'distance', 'isFlirtstar', 'isHighlighted',
                        'isInfluencer', 'isMobile', 'isNew', 'isOnline', 'isVip', 'lang_count',
                        'lang_fr', 'lang_en', 'lang_de', 'lang_it', 'lang_es', 'lang_pt', 'verified',
                        'shareProfileEnabled', 'lastOnlineDate', 'lastOnlineTime', 'birthd', 'crypt',
                        'freetext', 'whazzup', 'userId', 'isSystemProfile']

# Supprimer les colonnes spécifiées du DataFrame
dataset = dataset.drop(colonnes_a_supprimer, axis=1)

# Identificar duplicados excluyendo blank y cero
duplicates_to_drop = dataset[(dataset['pictureId'].notnull()) & (dataset['pictureId'] != '') & (dataset['pictureId'] != 0)]
duplicates_to_drop = duplicates_to_drop[duplicates_to_drop.duplicated(subset='pictureId', keep=False)]

# Eliminar las filas identificadas como duplicadas
dataset = dataset.drop(duplicates_to_drop.index)

# Calculer kisses_per_visit et arrondir à deux décimales, puis ajouter le symbole '%'
dataset['taux_aime'] = (dataset['counts_kisses'] / dataset['counts_profileVisits'] * 100).round(2)

# Discrétiser la colonne kisses par visite
dataset['aime_5_pct'] = pd.cut(dataset['taux_aime'], bins=[-float('inf'), 5, float('inf')], labels=['moins_5_pct_aime', 'plus_5_pct_aime'], right=False)

# Créer la nouvelle colonne 'visitas_mas_10' selon les conditions spécifiées
dataset['10_visites'] = dataset.apply(lambda row: 'plus_10_visites' if row['counts_profileVisits'] >= 10 else 'moins_10_visites', axis=1)
   
# Créer la nouvelle colonne 'compte_populaire' basée sur les conditions spécifiées
dataset['compte_populaire'] = dataset.apply(lambda row: 'populaire' if row['aime_5_pct'] == 'plus_5_pct_aime' and row['10_visites'] == 'plus_10_visites' else 'non_populaire', axis=1)
    
# Compter combien de fois "populaire" apparaît dans la colonne compte_populaire
count_populaire = (dataset['compte_populaire'] == 'populaire').sum()

# Normalizar la columna counts_profileVisits
dataset['norm_visites'] = (dataset['counts_profileVisits'] / dataset['counts_profileVisits'].max()).round(3)

# Normalizar la columna counts_profileVisits
dataset['norm_kisses'] = (dataset['counts_kisses'] / dataset['counts_kisses'].max()).round(3)

# Filtrar las filas donde counts_kisses sea menor o igual a counts_profileVisits
dataset = dataset[dataset['counts_kisses'] <= dataset['counts_profileVisits']]

# Afficher les premières lignes du DataFrame avec les nouvelles colonnes ajoutées
print(dataset.iloc[:5])

print("\n")

# Afficher le nombre de fois où "populaire" apparaît dans la colonne compte_populaire
print(f'Nombre de valeurs "populaire": {count_populaire}')

# Compter combien de fois "plus_10_visites" apparaît dans la colonne 10_visites
count_plus_10_visites = (dataset['10_visites'] == 'plus_10_visites').sum()

print(f'Nombre de "plus_10_visites": {count_plus_10_visites}')

# Compter combien de fois "plus_5_pct_aime" apparaît dans la colonne aime_5_pct
count_plus_5_pct_aime = (dataset['aime_5_pct'] == 'plus_5_pct_aime').sum()

print(f'Nombre de "plus_5_pct_aime": {count_plus_5_pct_aime}')




              counts_profileVisits  counts_kisses                 pictureId  \
name                                                                          
daeni                       8279.0          239.0  55303fc3160ba0eb728b4575   
italiana 92                  663.0           13.0  552e7b61c66da10d1e8b4c82   
Lauraaa                     1369.0           88.0  54a584ecc56da128638b4674   
Qqkwmdowlo                 22187.0         1015.0  54c92738076ea1b5338b4735   
schaessie {3               35262.0         1413.0  54e1a6f6c76da135748b4a3a   

              taux_aime        aime_5_pct       10_visites compte_populaire  \
name                                                                          
daeni              2.89  moins_5_pct_aime  plus_10_visites    non_populaire   
italiana 92        1.96  moins_5_pct_aime  plus_10_visites    non_populaire   
Lauraaa            6.43   plus_5_pct_aime  plus_10_visites        populaire   
Qqkwmdowlo         4.57  moins_5_pct_aime  plus_10_